# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper`.

You will find more info about virtual environments [here](../../2.python/1.python_fundamentals/02.Package-Managers/00.Package_Managers.md).

Activate it.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: ` https://country-leaders.herokuapp.com/status``
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [2]:
import requests

url = "https://country-leaders.herokuapp.com"
status_url = url + "/status"
req = requests.get(status_url)
print(req.text) if req.ok else print(req)

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [3]:
country_url = url + "/countries"
req = requests.get(country_url)
countries = requests.get(country_url).json()
print(req.text) if req.ok else print(req)
print(countries)


<Response [403]>
{'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [4]:
# Set the cookie_url variable (1 line)

# Query the enpoint, set the cookies variable and display it (3 lines)


Try to query the countries endpoint using the cookie, save the output and print it.

In [5]:
# query the /countries endpoint, assign the output to the countries variable (1 line)

# and check the status code and the countries variable's content (3 lines)


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [6]:
# query the /leaders endpoint, assign the output to the leaders variable (1 line)

# and check its content (4 lines)


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.herokuapp.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [7]:
# Set the leaders_url variable (1 line)

# query the /leaders endpoint using cookies and parameters (take any country in countries)
# check the result (3 lines) 


### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each entry in a dictionary called `leaders_per_country` like this one: 
`{'fr':[{leader0}, {leader1},...],
'en':[{leader0}, {leader1},...],
...
}`.

In [8]:
# 5 lines


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [9]:
import requests

def get_leaders():
    leaders_dict = {}
    url = "https://country-leaders.herokuapp.com"
    countries_u, cookies_u, leaders_u  = url + "/countries", url + "/cookie", url + "/leaders" 
    cookie = requests.get(cookies_u).cookies
    countries = requests.get(countries_u, cookies=cookie).json()
    for country in countries :
        param_country = "country=" + country
        leaders = requests.get(leaders_u,cookies=cookie,params=param_country).json()
        for leader in leaders:
            leaders_dict.setdefault(country, []).append(leader)
    return leaders_dict


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [10]:
print(get_leaders())


{'ma': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [11]:
req = requests.get("https://en.wikipedia.org/wiki/John_F._Kennedy").text
#reqtxt = req.text

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function to take a look. You will start the actual parsing in the next step.

In [12]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(req, 'html.parser')
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   John F. Kennedy - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"f1169331-da0f-4dc8-b5fd-849de1ae3f72","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"John_F._Kennedy","wgTitle":"John F. Kennedy","wgCurRevisionId":1109901604,"wgRevisionId":1109901604,"wgArticleId":5119376,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Japanese-language sources (ja)","All articles with dead external links","Articles with dead external links from April 2019","

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [51]:
texte = []
for paragraph in soup.find_all('p'):
    texte.append(paragraph.text)
print(texte[1])

John Fitzgerald Kennedy (May 29, 1917 – November 22, 1963), often referred to by his initials JFK and also known as Jack Kennedy, was an American politician who served as the 35th president of the United States from 1961 until his assassination near the end of his third year in office. Kennedy was the youngest person to assume the presidency by election. He was also the youngest president at the end of his tenure.[2] Kennedy served at the height of the Cold War, and the majority of his work as president concerned relations with the Soviet Union and Cuba. A Democrat, he represented Massachusetts in both houses of the U.S. Congress prior to his presidency.



If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [14]:
paragraphe = ''
for i in range(len(text)) :
    paragraphe += text[i]
    if text[i] == '.' and text[i+1] == '\n':
        break
print(paragraphe)
    


John Fitzgerald Kennedy (May 29, 1917 – November 22, 1963), often referred to by his initials JFK and also known as Jack Kennedy, was an American politician who served as the 35th president of the United States from 1961 until his assassination near the end of his third year in office. Kennedy was the youngest person to assume the presidency by election. He was also the youngest president at the end of his tenure.[2] Kennedy served at the height of the Cold War, and the majority of his work as president concerned relations with the Soviet Union and Cuba. A Democrat, he represented Massachusetts in both houses of the U.S. Congress prior to his presidency.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [55]:
# 10 lines
def get_first_paragraph(wikipedia_url):
   print(wikipedia_url) # keep this for the rest of the notebook
   soup = BeautifulSoup(requests.get(wikipedia_url).text, 'html.parser')
   soup.prettify()
   texte = []
   for paragraph in soup.find_all('p'):
      texte.append(paragraph.text)
   return texte[1]

print(get_first_paragraph("https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere"))

https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere
Hij was een zoon van Jean-Baptiste van de Vyvere en van Marie-Valérie Sierens en trouwde in 1894 met Marie-Louise de Neeff (1870-1926). Hij hertrouwde in 1931 met Nydia Merten (1910-1995). Uit het eerste huwelijk had hij vijf kinderen en nog een dochter uit het tweede. In 1931 werd hij in de Belgische erfelijke adel opgenomen met de bij eerstgeboorte overdraagbare titel van burggraaf.



### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [16]:
# 4 lines


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [17]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [18]:
# < 20 lines


### Caching
Good job! This function will be called many times. First because there are many leaders to loop through. Second because you are developing your code and are likely to re-run your loop multiple times. Third because the API might change and contain new leaders and you might want to update your data periodically.

For all these reasons, it can be benificial to *cache* the result of the `get_first_paragraph()` function so that if it called with the same parameter, you expect the same output and thus do not need to query Wikipedia another time.

You will use the [lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache) decorator from the `functools` module. It is a tiny code change that will speed things up big time. You can use a `maxsize` of None. If you have questions about decorators, you will find more information [here](../../2.python/2.python_advanced/07.Decorator/decorators.ipynb)

In [19]:
# < 20 lines


Let's check the speedup!

In [20]:
# Just run the cell. 
%%timeit
get_first_paragraph(wikipedia_url)

UsageError: Line magic function `%%timeit` not found.


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [21]:
# 19 lines


In [22]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function to check if the `status_code` is a cookie error. In which case, get new ones and query the api again.

If not, you may have a fast connection or a bit of luck. Do it anyway.

In [23]:
# 22 lines


Check the output of your function again.

In [24]:
%%time
# Check the output of your function (1 line)


CPU times: total: 0 ns
Wall time: 0 ns


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [25]:
# <20 lines

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [26]:
# 22 lines


Test your new functions.



In [27]:
# Just run it.
%%time
leaders_per_country = get_leaders()

UsageError: Line magic function `%%time` not found.


Using a Session makes the first call to `get_leaders()` faster however, for technical reasons, the `@lru_cache` decorator no longer works. 

Problem: The `session` parameter is not *hashable* and since the cache is a dictionary using all the parameters as key, this poses a problem.

Solution: Create our own cache that only cares about the url.

The code for this is provided. You should only read it.

In [28]:
cache = {}
def hashable_cache(f):
    def inner(url, session):
        if url not in cache:
            cache[url] = f(url, session)
        return cache[url]
    return inner

Overwrite the `get_first_paragraph()` function to use the new decorator like this: `@hashable_cache`.


In [29]:
# <20 lines

Check that the cache is working again.

In [30]:
# Just run it.
%%time
leaders_per_country = get_leaders()

UsageError: Line magic function `%%time` not found.


## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Make a tour in our [file handling section](../../2.python/2.python_advanced/04.File-handling/file_handling.ipynb) to know more about how to read and write files using Python.

In [31]:
# 3 lines


Make sure the file can be read back. Write the code to read the file.

In [32]:
#3 lines

Make a function `save()` to call this code easily.

In [33]:
# 3 lines


In [34]:
# Call the function (1 line)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!